In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

FinancialLabel = "Financial/Job Problem"
Legallabel = "Legal Problem"
SchoolLabel = "School or Academic Related Problem"
HealthLabel = "Lack of Access to Health/Mental Health Care"
FinancialPredicted = "Financial - Predicted"
LegalPredicted = "Legal - Predicted"
SchoolPredicted = "School - Predicted"
HealthPredicted = "Health - Predicted"

In [2]:

#scores = pd.read_csv('ben_dataset_all_class_scores_embeddings_using_first_384_words.csv')
#bendata = pd.read_csv('/Users/minijain/Downloads/SocWebLab/Structural_Violence/Final Ben dataset labelled.csv')
bendata = pd.read_csv('/Users/minijain/Downloads/SocWebLab/Structural_Violence/finalbendatasetwithpreds.csv')
nelagt = pd.read_csv('Transfer/labels_NELAGT.csv')
countydata = pd.read_csv('Transfer/County_data_NELALocal.csv')

In [393]:
result = bendata
print(result.shape)

In [395]:
result.loc[result['Publisher'] == "chicagosuntimes", 'Publisher'] = "chicagosun-times"
result.loc[result['Publisher'] == "wakingtimes", 'Publisher'] = "thewakingtimes"
requiredpublishers = result['Publisher'].unique()
allregionalpublishers = countydata['source']
allnationalpublishers = nelagt['source']

common_elements = set(allnationalpublishers).intersection(allregionalpublishers)
all_elements = set(allnationalpublishers).union(allregionalpublishers)

for i in requiredpublishers:
    if i not in all_elements:
        print(i)

hitandrun
pamelagellerreport
skynewsus
bonginoreport
freedom-bunker
csmonitor


In [397]:
result = result[(result['Reasoning'] != "TooManyTokens") & (result['Reasoning'] != "NotSuicide")]
result = result.drop([FinancialLabel, SchoolLabel, Legallabel, HealthLabel, 'Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'Comments', 'None', 'Error'], axis = 1)
print(result.shape)

(13629, 15)


### BREAK

In [399]:
nationalsources = nelagt[nelagt['country'] == 'USA']['source'].unique()
regionalsources = countydata['source'].unique()
print(len(nationalsources))
print(len(regionalsources))

common_elements = [element for element in nationalsources.tolist() if element in regionalsources.tolist()]
print(common_elements)

213
315
['chicagosun-times', 'dailyherald']


In [400]:
def national_func(row):
    val = row['Publisher']
    #print(val)
    if row['Publisher'] in nationalsources:
        return 1
    else:
        return 0
def regional_func(row):
    val = row['Publisher']
    #print(val)
    if row['Publisher'] in regionalsources:
        return 1
    else:
        return 0

# Apply the function to create the new column
result['National'] = result.apply(national_func, axis=1)
result['Regional'] = result.apply(regional_func, axis=1)

In [401]:
print("National articles: ", len(result[result['National'] == 1]))
print("Regional articles: ", len(result[result['Regional'] == 1]))
print("None articles: ", len(result[(result['National'] == 0) & (result['Regional'] == 0)]))

National articles:  8891
Regional articles:  4426
None articles:  573


In [402]:
regionaldf = result[result['Regional'] == 1]
nationaldf = result[result['National'] == 1]

In [403]:
regionaldf.to_csv('regional.csv')
nationaldf.to_csv('national.csv')

In [409]:
nelagt['bias'].unique()
leftsources = nelagt[(nelagt['country'] == 'USA') &  ((nelagt['bias'] == 'left-center') | (nelagt['bias'] == 'left') | (nelagt['bias'] == 'left-bias') )]['source'].unique()
rightsources = nelagt[(nelagt['country'] == 'USA') &  ((nelagt['bias'] == 'right-center') | (nelagt['bias'] == 'right') | (nelagt['bias'] == 'right-bias') )]['source'].unique()
print(len(leftsources))
print(len(rightsources))
def left_func(row):
    val = row['Publisher']
    #print(val)
    if row['Publisher'] in leftsources:
        return 1
    else:
        return 0
def right_func(row):
    val = row['Publisher']
    #print(val)
    if row['Publisher'] in rightsources:
        return 1
    else:
        return 0

# Apply the function to create the new column
result['Left'] = result.apply(left_func, axis=1)
result['Right'] = result.apply(right_func, axis=1)

58
32


In [410]:
policticalleaningadded = pd.merge(nationaldf, nelagt, left_on='Publisher', right_on='source', how='inner')
leftbias = policticalleaningadded[(policticalleaningadded['bias'] == 'left') | (policticalleaningadded['bias'] == 'left-center')]
rightbias = policticalleaningadded[(policticalleaningadded['bias'] == 'right') | (policticalleaningadded['bias'] == 'right-center')]
nobias = policticalleaningadded[policticalleaningadded['bias'] == 'center']

In [411]:
leftbias.to_csv('leftbias.csv')
rightbias.to_csv('rightbias.csv')

In [ ]:
# seed topics
SEED_TOPICS_FLAG = False # seeds topics that model will attempt to converge to
if SEED_TOPICS_FLAG:
    SEED_TOPICS = [['learning', 'learn', 'knowledge'],['curious', 'curiosity'],['trends', 'latest', 'abreast', 'technology'],['skills', 'improve', 'skillset', 'practice', 'career'],['datasets', 'access', 'public', 'available'],['competition, competetive'],['money', 'win', 'prizes'],['fun','hobby', 'enjoy', 'love']]
else:
    SEED_TOPICS = [[]]
    
# umap params
UMAP_N_NEIGHBOURS = 20
UMAP_N_COMPONENTS = 15
RANDOM_SEED = 42

# hdbscan params
MIN_CLUSTER_SIZE = 15

# ctfidf params
#REDUCE_FREQUENT_WORDS = False
REDUCE_FREQUENT_WORDS = True

# vectorizer params
STOP_WORDS_FLAG = True # stops words being used in topics, but not for embeddings
NGRAM_RANGE = (1,2)  

if STOP_WORDS_FLAG:
    STOP_WORDS = stopwords.words('english')
else: 
    STOP_WORDS = []

# representation model params
DIVERSITY = 0.1 # val in [0,1] specifying diversity of words in each topic

# outlier reduction params
OUTLIER_THRESH = 0.2

PRETRAINED_SENTENCE_MODEL = 'all-MiniLM-L12-v2'

In [ ]:
sentence_model = SentenceTransformer(PRETRAINED_SENTENCE_MODEL)
embeddings = sentence_model.encode(comments_all, show_progress_bar=True)